In [1]:
from decouple import config
from bs4 import BeautifulSoup as soup
import urllib.request as reqs
import pickle
import requests

In [2]:
def get_sentiment(reviews):
    sentiments=[]
    vectorizer = pickle.load(open('../pickle_modules/transform.pkl', 'rb'))
    arr = vectorizer.transform(reviews)
    clf = pickle.load(open('../pickle_modules/mnb.pkl', 'rb'))
    sentiments = clf.predict(arr)
    return sentiments

In [3]:
def get_movie_info(tmdb_id):
    response=requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(tmdb_id,config('API_KEY')))
    info=response.json()
    return info

In [16]:
get_movie_info(19995)

{'adult': False,
 'backdrop_path': '/jlQJDD0L5ZojjlS0KYnApdO0n19.jpg',
 'belongs_to_collection': {'id': 87096,
  'name': 'Avatar Collection',
  'poster_path': '/gC3tW9a45RGOzzSh6wv91pFnmFr.jpg',
  'backdrop_path': '/syGPZuzcHBBHMLiNDN0x0Tms4Fk.jpg'},
 'budget': 237000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'https://www.avatar.com/movies/avatar',
 'id': 19995,
 'imdb_id': 'tt0499549',
 'original_language': 'en',
 'original_title': 'Avatar',
 'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
 'popularity': 352.589,
 'poster_path': '/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg',
 'production_companies': [{'id': 444,
   'logo_path': '/42UPdZl6B2cFXgNUASR8hSt9mpS.png',
   'name': 'Dune Entertainment',
   'origin_country': 'US'},
  {'id

In [4]:
def get_reviews_and_sentiment(imdb_id):
    # sauce=reqs.urlopen('https://www.imdb.com/title/{}/criticreviews'.format(get_imdb_id(tmdb_id))).read()
    sauce=reqs.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read()
    page_soup=soup(sauce,'lxml')
    # results=page_soup.find_all("div", attrs={"class": "summary","itemprop":"reviewbody"})
    results=page_soup.find_all("div", {"class": "text show-more__control"})
    reviews_list=[]
    i=0
    for review in results:
        reviews_list.append(review.text)
        i=i+1
        if i>=5:
            break
    
    reviews_status=get_sentiment(reviews_list)
    movie_reviews = {reviews_list[i]: reviews_status[i]
                     for i in range(len(reviews_list))}
    return movie_reviews

In [5]:
def get_all_details(tmdb_id):
    info = get_movie_info(tmdb_id)
    movie_dict = {'imdb_id': info['imdb_id'],
                  "title": info['title'],
                  "image": 'https://image.tmdb.org/t/p/original'+info['backdrop_path'],
                  "overview": info['overview'],
                  "votes": info['vote_average'],
                  "year":info['release_date'].split()[0],
                  get_reviews_and_sentiment(info['imdb_id'])}
    return movie_dict

In [8]:
details=get_all_details(19995)

In [13]:
details['reviews'].values()

dict_values([1, 1, 1, 1, 1])